In [4]:
import pandas as pd
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

**Import the dataset**

In [23]:
data = pd.read_csv('movie_review.csv', delimiter=',')
data

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos
...,...,...,...,...,...,...
64715,9,cv999,14636,20,that lack of inspiration can be traced back to...,neg
64716,9,cv999,14636,21,like too many of the skits on the current inca...,neg
64717,9,cv999,14636,22,"after watching one of the "" roxbury "" skits on...",neg
64718,9,cv999,14636,23,"bump unsuspecting women , and . . . that's all .",neg


In [24]:
df = data[['text', 'tag']]
df

,text,tag
0,films adapted from comic books have had plenty...,pos
1,"for starters , it was created by alan moore ( ...",pos
2,to say moore and campbell thoroughly researche...,pos
3,"the book ( or "" graphic novel , "" if you will ...",pos
4,"in other words , don't dismiss this film becau...",pos
...,...,...
64715,that lack of inspiration can be traced back to...,neg
64716,like too many of the skits on the current inca...,neg
64717,"after watching one of the "" roxbury "" skits on...",neg
64718,"bump unsuspecting women , and . . . that's all .",neg


**Pre-processing des données textuelles**

In [25]:
import re

def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U0001F004-\U0001F0CF"  # Miscellaneous Symbols and Pictographs
                           u"\U0001F0D0-\U0001F0FF"  # Emoji for Legacy Computing
                           u"\U0001F1E0-\U0001F1FF"  # Regional Indicator Symbols
                           u"\U0001F200-\U0001F251"  # Enclosed Alphanumeric Supplement
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def preprocess_text(text):
   # Remove emojis
    text_removed_emojis = remove_emojis(text)
    # Remove punctuation
    text_removed_emojis_ponc  = ''.join([char for char in text_removed_emojis if char not in string.punctuation])

    # Tokenization
    words = word_tokenize(text_removed_emojis_ponc)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words_without_sw = [word for word in words if word.lower() not in stop_words]

    # Return the preprocessed text
    return words_without_sw
  #Application de preprocessing
df['cleaned_text'] = df['text'].apply(preprocess_text)

<ipython-input-25-3c4c77043310>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_text'] = df['text'].apply(preprocess_text)


In [26]:
df.head()

,text,tag,cleaned_text
0,films adapted from comic books have had plenty...,pos,"[films, adapted, comic, books, plenty, success..."
1,"for starters , it was created by alan moore ( ...",pos,"[starters, created, alan, moore, eddie, campbe..."
2,to say moore and campbell thoroughly researche...,pos,"[say, moore, campbell, thoroughly, researched,..."
3,"the book ( or "" graphic novel , "" if you will ...",pos,"[book, graphic, novel, 500, pages, long, inclu..."
4,"in other words , don't dismiss this film becau...",pos,"[words, dont, dismiss, film, source]"


**Entraînement du modèle Word2Vec en utilisant Gensim. Le
modèle détermine comment représenter les mots sous forme de vecteurs en fonction de leur
utilisation dans le dataset.
on teste sur le mot *'word'* :**

In [27]:
from gensim.models import Word2Vec
# Liste des phrases prétraitées (liste de listes de mots)
sentences = df['cleaned_text'].tolist()

# Créez et entraînez le modèle Word2Vec
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=0)
#le vecteur du mot 'word'
vector = model.wv['word']
vector

array([-0.3654503 ,  0.39609116,  0.15436678,  0.10467533,  0.09124174,
       -1.1599439 ,  0.608853  ,  1.2638223 , -0.4984577 , -0.6293415 ,
       -0.30965713, -0.9743413 , -0.0709009 ,  0.32202676,  0.37222618,
       -0.5300546 ,  0.06329609, -0.87069935,  0.05085915, -1.2321947 ,
        0.5032505 ,  0.22488686,  0.57207847, -0.43223596, -0.22839643,
        0.11106104, -0.5619277 , -0.25490192, -0.6946655 ,  0.06276634,
        0.45654383,  0.28671855,  0.09230621, -0.47316483, -0.197697  ,
        0.85642695,  0.24391972, -0.18272172, -0.36269796, -0.9894009 ,
        0.49131516, -0.6511271 , -0.34059492,  0.05261976,  0.43505934,
       -0.31751198, -0.47056377, -0.01793153,  0.2528454 ,  0.34578094,
        0.2671006 , -0.4166676 , -0.44956493, -0.0881563 , -0.18548933,
        0.14275669,  0.56053036, -0.00742007, -0.6897501 ,  0.4843575 ,
       -0.08914667,  0.25948423, -0.23023696,  0.09699611, -0.76487565,
        0.58878744,  0.21031326,  0.31539002, -0.79826987,  0.70

**Vectorisation des reviews de movies :
représentation de chaque review d’un film sous
forme de vecteur en prenant la moyenne de Word2Vec embeddings des mots. Cela crée une
représentation vectorielle de chaque review qui montre comment les mots s&#39;articulent et ce qu&#39;ils
signifient.**

In [28]:

import numpy as np

def average_word_vectors(words, model, num_features):
    feature_vector = np.zeros((num_features,), dtype="float32")
    nwords = 0

    for word in words:
        if word in model.wv.key_to_index:
            nwords = nwords + 1
            feature_vector = np.add(feature_vector, model.wv[word])

    if nwords > 0:
        feature_vector = np.divide(feature_vector, nwords)

    return feature_vector


In [29]:
# Exemple d'application de la fonction de vectorisation
#100 : vector_size
df['vectorized_review'] = df['cleaned_text'].apply(lambda x: average_word_vectors(x, model, 100))


<ipython-input-29-eb3a319aa34b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vectorized_review'] = df['cleaned_text'].apply(lambda x: average_word_vectors(x, model, 100))


In [30]:
df.head()

,text,tag,cleaned_text,vectorized_review
0,films adapted from comic books have had plenty...,pos,"[films, adapted, comic, books, plenty, success...","[-0.3423267, 0.38561714, 0.20574531, 0.1120308..."
1,"for starters , it was created by alan moore ( ...",pos,"[starters, created, alan, moore, eddie, campbe...","[-0.2602618, 0.181174, 0.17263673, 0.10084496,..."
2,to say moore and campbell thoroughly researche...,pos,"[say, moore, campbell, thoroughly, researched,...","[-0.2943042, 0.42798555, 0.18957858, 0.0379115..."
3,"the book ( or "" graphic novel , "" if you will ...",pos,"[book, graphic, novel, 500, pages, long, inclu...","[-0.28735775, 0.18494804, 0.1655903, 0.1014511..."
4,"in other words , don't dismiss this film becau...",pos,"[words, dont, dismiss, film, source]","[-0.34892672, 0.3458081, 0.25621024, 0.2260077..."


**Division des données :
Divisez le dataset en ensembles d&#39;entraînement (training sets) et de test (testing sets).**

In [31]:
from sklearn.model_selection import train_test_split

# Séparez les données en X (caractéristiques) et y (étiquettes)
X = df['vectorized_review']
y = df['tag']

# Divisez les données en ensembles d'entraînement (80%) et de test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


**Construction d&#39;un classificateur :
Entraînez un modèle de machine learning en utilisant la logistic regression,
support vector machines en utilisant les représentations vectorielles des reviews comme input
features et les étiquettes de sentiment comme variable cible (label).**

In [32]:
from sklearn.linear_model import LogisticRegression

# Exemple avec la régression logistique
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train.tolist(), y_train)


LogisticRegression(max_iter=1000)

**Évaluation du modèle :
utilisation des métriques comme accuracy, precision, recall, et F1-score
pour évaluer les performances sur l&#39;ensemble de test.**

In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Prédiction sur l'ensemble de test
y_pred = classifier.predict(X_test.tolist())

# Calcul des métriques
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Affichage des résultats
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 score: {f1}')


Accuracy: 0.5751699629171817
Precision: 0.5769984528942506
Recall: 0.5751699629171817
F1 score: 0.5700708523475512
